In [1]:
import dask.bag as db
import json
from dask.distributed import Client
import dask.dataframe as dd
import xarray as xr

In [ ]:
client = Client()
client.dashboard_link

In [23]:
import bz2
from typing import Iterator

def decompress_partition(blocks: Iterator[bytes]) -> Iterator[str]:
    dec = bz2.BZ2Decompressor()                # Stream‐friendly decompressor :contentReference[oaicite:8]{index=8}
    buffer = b""
    for chunk in blocks:
        buffer += dec.decompress(chunk)        # chunk is guaranteed bytes
        parts = buffer.split(b"\n")
        buffer = parts.pop()                   # keep the trailing partial line
        for line in parts:
            yield line.decode("utf-8")
    if buffer:
        yield buffer.decode("utf-8")


In [ ]:
import json
import dask
import dask.bag as db
import dask.bytes as dbbytes

# 1. Read raw BZ2 bytes (~1 MiB blocks)
compressed_blocks, _ = dbbytes.read_bytes(
    "data/ping-2025-06-10T0000.bz2",
    blocksize="1 MiB",
    compression=None
)

bag_blocks = db.from_sequence(compressed_blocks, partition_size=1)

# 2. Wrap each block in the streaming decompressor
lines = bag_blocks.map_partitions(decompress_partition)  # Returns a Bag of text lines :contentReference[oaicite:9]{index=9}

# 3. Build a Bag from these delayed lists of lines
bag = db.from_delayed(lines)

# 4. Parse JSON and filter by your timestamp
records  = bag.map(json.loads)
filtered = records.filter(lambda rec: rec.get("sent", -1) == 3)

In [ ]:
filtered.take(2)

In [ ]:
pi

In [30]:
df.to_parquet("data/ping-2025-06-10-0h-lat", engine="pyarrow")